# CR3BP low-thrust

In this tutorial we show the use of the {class}`pykep.trajopt.direct_cr3bp` to find a low-thrust (zero-order hold continuous thrust) trajectory connecting two orbits in the circular restricted 3-body problem (CR3BP). 

The decision vector for this class, compatible with pygmo {cite:p}`pagmo` UDPs (User Defined Problems), is:

$$
\mathbf x = [t_0, m_f, V_{sx}^\infty, V^\infty_{sy}, V^\infty_{sz}, V^\infty_{fx}, V^\infty_{fy}, V^\infty_{fz}, u_{x0}, u_{y0}, u_{z0}, u_{x1}, u_{y1}, u_{z1}, ..., T_{tof}]
$$

containing the starting epoch $t_0=0$, the final mass $m_f$ as well as the starting and final $V^{\infty}$, throttles and the time-of-flight $T_{tof}$.

:::{note}
This notebook makes use of the commercial solver SNOPT 7 and to run needs a valid `snopt_7_c` library installed in the system. In case SNOPT7 is not available, you can still run the notebook using, for example `uda = pg.algorithm.nlopt("slsqp")` with minor modifications.

Basic imports:

In [1]:
import pykep as pk
import numpy as np
import time
import pygmo as pg
import pygmo_plugins_nonfree as ppnf
import time

from matplotlib import pyplot as plt

We start defining the problem data. For the purpose of this simple notebook we choose a simple Earth to Mars transfer.

In [2]:
import json

# Load JSON file
# with open("/Users/harry.holt/Documents/PostDoc_ACT/GTOC/benchmark-astrodynamics-sqp-scp/cr3bp_fixed_time_rdv/instances.json", "r") as f:
with open("/Users/harry.holt/Documents/PostDoc_ACT/GTOC/benchmark-astrodynamics-sqp-scp/cr3bp_fixed_time_rdv/instances_lb.json", "r") as f:
    data = json.load(f)

# Id to load
ID = 0

# Problem data
mu = pk.MU_MOON / (pk.MU_EARTH + pk.MU_MOON)

# Example masses (you can adjust logic if they differ)
ms = 1500.0
mf = 1300.0
isp = 3000  # fixed across cases

for k, v in data.items():
    if int(k) == int(ID):
        max_thrust = v["umax"]
        tof = v["tspan"][1]
        veff = isp * pk.G0
        
        # Initial state
        x0 = v["x0"]
        rs = np.array(x0[0:3])
        vs = np.array(x0[3:6])
        
        # Final state
        xf = v["xf"]
        rf = np.array(xf[0:3])
        vf = np.array(xf[3:6])


In [3]:
# Throttles and cuts
nseg = 2
cut = 0.6

In [4]:
# Normalise
L = 384400e3
TIME = np.sqrt(L**3 / (pk.MU_EARTH+pk.MU_MOON))
VEL = L / TIME
ACC = VEL / TIME
MASS = ms

print(f'Normalise L {L:.4f} T {TIME:.4f} V {VEL:.4f} M {MASS:.4f}')

# Problem data
# mu already in non-dimensional units
# max_thrust = max_thrust / (MASS * L**3 / TIME**2)
veff = veff / VEL

# Initial state
ms = ms / MASS
# rs already in non-dimensional units
# vs already in non-dimensional units

# Final state
mf = mf / MASS
# rf already in non-dimensional units
# vf already in non-dimensional units

# tof
# tof = tof / TIME
# tof already in non-dimensional units

Normalise L 384400000.0000 T 375190.2619 V 1024.5468 M 1500.0000


Convert final position and velocity into an orbit

In [5]:
posvel0 = [
    rs.tolist(),
    vs.tolist()
]

# Propagate backwards to t0
ta = pk.ta.get_cr3bp(1e-16)
ta.pars[0] = mu
ta.time = tof
ta.state[:6] = rf.tolist() + vf.tolist()

tgrid = np.linspace(tof,0.0,10)
sol = ta.propagate_grid(tgrid)
integration = sol[5]

rf_0 = integration[-1,:3]
vf_0 = integration[-1,3:6]

posvelf = [
    rf_0.tolist(),
    vf_0.tolist()
]

print(f'IC {rf_0} {vf_0} t {ta.time}')
print(f'FC {rf} {vf} {tof}')


IC [ 1.01728518 -0.02281703 -0.16695631] [-0.03029824 -0.08797148  0.12306713] t 1.3010426069826053e-18
FC [0.98740797 0.         0.00798274] [0.         1.71563544 0.        ] 5.0


We here instantiate two different versions of the same UDP (User Defined Problem), with analytical gradients and without. 

For the purpose of this simple notebook we choose a relatively simple Earth to Mars transfer with an initial $V_{\infty}$ of 3 km/s.

In [6]:
udp_g = pk.trajopt.direct_cr3bp(
        pls=posvel0,
        plf=posvelf,
        ms=ms,
        mu=mu,
        max_thrust=max_thrust,
        veff=veff,
        t0_bounds=[0.0, 0.0],
        tof_bounds=[tof,tof],
        mf_bounds=[ms*0.5, ms],
        vinfs=0.,
        vinff=0.,
        nseg=nseg,
        cut=cut,
        with_gradient=True,
        high_fidelity=True
)

## Analytical performances of the analytical gradient

And we take a quick look at the performances of the analytical gradient with respect to the numerically computed one.

In [7]:
# We need to generste a random chromosomes compatible with the UDP where to test the gradient.
prob_g = pg.problem(udp_g)
pop_g = pg.population(prob_g, 1)

In [ ]:
def compute_numerical_gradient(sf_leg, sf_leg_type = 'lf'):
    import numpy as np
    import pykep as pk
    import pygmo as pg

    state_length = np.array(sf_leg.rvs).flatten().size + 1
    throttle_length = np.array(sf_leg.throttles).size
    chromosome = np.zeros((state_length * 2 + throttle_length + 1))
    chromosome[0:state_length] = np.append(np.array(sf_leg.rvs).flatten(), sf_leg.ms)
    chromosome[state_length:state_length+throttle_length] = np.array(sf_leg.throttles)
    chromosome[state_length+throttle_length:state_length*2+throttle_length] = np.append(np.array(sf_leg.rvf).flatten(), sf_leg.mf)
    chromosome[-1] = sf_leg.tof

    def set_and_compute_constraints(chromosome, sf_leg_type = 'lf'):

        if sf_leg_type == 'hf' or sf_leg_type == 'high-fidelity':
            sf_leg_constraint = pk.leg.sims_flanagan_hf_nd()
            # sf_leg_constraint = pk.leg.sims_flanagan_hf_nd(
            #     tas = pk.ta.get_zero_hold_cr3bp(1e-16),
            #     tas_var = pk.ta.get_zero_holdhold_cr3bp_var(1e-16)
            # )
        else:
            xxx
            sf_leg_constraint = pk.leg.sims_flanagan()
        sf_leg_constraint.cut = 0.5
        sf_leg_constraint.max_thrust = 1
        sf_leg_constraint.mu = 0.012
        sf_leg_constraint.veff = 1
        sf_leg_constraint.rvs = [chromosome[0:3],chromosome[3:6]]
        sf_leg_constraint.ms = chromosome[6]
        sf_leg_constraint.throttles = chromosome[state_length:state_length+throttle_length]
        sf_leg_constraint.rvf = [chromosome[state_length+throttle_length:state_length+throttle_length+3],chromosome[state_length+throttle_length+3:state_length+throttle_length+6]]
        sf_leg_constraint.mf = chromosome[2*state_length+throttle_length-1]
        sf_leg_constraint.tof = chromosome[2*state_length+throttle_length]
        eq_con = sf_leg_constraint.compute_mismatch_constraints()
        ineq_con = sf_leg_constraint.compute_throttle_constraints()
        return np.concatenate((eq_con, ineq_con))

    return pg.estimate_gradient_h(callable = lambda x : set_and_compute_constraints(x, sf_leg_type), x=chromosome)

In [ ]:
def test_mc_grad_hf():
    import numpy as np

    sf_leg = pk.leg.sims_flanagan_hf_nd()
    # sf_leg = pk.leg.sims_flanagan_hf_nd(
    #     tas = pk.ta.get_zero_hold_cr3bp(1e-16),
    #     tas_var = pk.ta.get_zero_hold_cr3bp_var(1e-16)
    # )
    sf_leg.cut = 0.5
    sf_leg.throttles = np.array([0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24,
    0.20, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34])
    sf_leg.rvs = np.array([[1, 0.1, -0.1], [0.2, 1.0, -0.2]])
    sf_leg.ms = 1
    sf_leg.rvf = np.array([[1.2, -0.1, 0.1], [-0.2, 1.023, -0.44]])
    sf_leg.mf = 13 / 15
    sf_leg.max_thrust = 1
    sf_leg.mu = 0.012
    sf_leg.veff = 1
    sf_leg.tof = 1
    state_length = np.array(sf_leg.rvs).flatten().size + 1
    throttle_length = np.array(sf_leg.throttles).size

    num_grad = compute_numerical_gradient(sf_leg, sf_leg_type = 'hf')
    num_grad = num_grad.reshape((17, 45), order='C')
    grad_rvm, grad_rvm_bck, grad_final = sf_leg.compute_mc_grad()
    a_tc_grad = sf_leg.compute_tc_grad()
    a_grad = np.zeros((state_length+throttle_length // 3, 2 * state_length + throttle_length + 1))
    a_grad[0:state_length, 0:state_length] = grad_rvm
    a_grad[0:state_length, state_length:state_length + throttle_length] = grad_final[:,0:throttle_length] 
    a_grad[0:state_length, state_length+throttle_length:state_length*2+throttle_length] = grad_rvm_bck
    a_grad[0:state_length, state_length*2+throttle_length] = grad_final[:, throttle_length:throttle_length + 1].reshape(7,)
    a_grad[state_length:, state_length:state_length+throttle_length] = a_tc_grad
    return np.allclose(num_grad, a_grad, atol=1e-8)

In [10]:
test_mc_grad_hf()

True

Compare gradient accuracy

In [11]:
# Check gradientss
# Analytical
an_grad = udp_g.gradient(pop_g.champion_x)
# Numerical
num_grad = pg.estimate_gradient_h(udp_g.fitness, pop_g.champion_x, dx=1e-12)

# Suppose sparsity gives the positions (flat indices in num_grad)
sparsity = udp_g.gradient_sparsity()  # e.g. [0, 3, 10, ...]

# Make an empty dense gradient
dim = int(9+3*nseg)
dense_ana_grad = np.zeros_like(num_grad).reshape(int(len(num_grad)/dim),dim)
num_grad = num_grad.reshape(int(len(num_grad)/dim),dim)

# Fill in analytical entries at the right positions
for jj in range(len(an_grad)):
    dense_ana_grad[sparsity[jj][0], sparsity[jj][1]] = an_grad[jj]
    diff_tmp = abs(dense_ana_grad[sparsity[jj][0], sparsity[jj][1]] - num_grad[sparsity[jj][0], sparsity[jj][1]])
    if diff_tmp > 1:
        print('sparsity[jj]',sparsity[jj] , 'analytical', dense_ana_grad[sparsity[jj][0], sparsity[jj][1]],'numerical', num_grad[sparsity[jj][0], sparsity[jj][1]],'diff',diff_tmp)
# dense_ana_grad = dense_ana_grad.reshape(-1)
# num_grad = num_grad.reshape(-1)

# Now you can compare
diff = num_grad - dense_ana_grad
# print('Analytical', dense_ana_grad)
# print('Numerical', num_grad)
print('diff', np.shape(diff))
print("‖diff‖:", np.linalg.norm(diff))

sparsity[jj] [1, 0] analytical 36.984684314417365 numerical 34.70503884273057 diff 2.2796454716867913
sparsity[jj] [1, 8] analytical 3.4429946128799402 numerical 10.329574233007104 diff 6.886579620127164
sparsity[jj] [1, 11] analytical -3.2250427622350157 numerical -9.675475235818947 diff 6.450432473583931
sparsity[jj] [2, 9] analytical 3.4515887990943743 numerical 10.356841310491898 diff 6.905252511397524
sparsity[jj] [2, 12] analytical -3.209795087899189 numerical -9.629630426388758 diff 6.419835338489569
sparsity[jj] [3, 10] analytical 3.502661888747291 numerical 10.507002675315865 diff 7.004340786568575
sparsity[jj] [3, 13] analytical -3.1843605786315448 numerical -9.553232279320886 diff 6.368871700689342
sparsity[jj] [4, 8] analytical 2.900354892111053 numerical 8.701298940631357 diff 5.800944048520304
sparsity[jj] [4, 11] analytical 2.5560075915568015 numerical 7.6680809849942015 diff 5.1120733934374005
sparsity[jj] [5, 9] analytical 2.911657146947703 numerical 8.7360193153548 di

In [ ]:
pk.ta.zero_hold_dyn()

[(x, vx),
 (y, vy),
 (z, vz),
 (vx,
  (((-p0 / (x**2.0000000000000000 + y**2.0000000000000000 + z**2.0000000000000000)**1.5000000000000000) * x) + (p2 / m))),
 (vy,
  (((-p0 / (x**2.0000000000000000 + y**2.0000000000000000 + z**2.0000000000000000)**1.5000000000000000) * y) + (p3 / m))),
 (vz,
  (((-p0 / (x**2.0000000000000000 + y**2.0000000000000000 + z**2.0000000000000000)**1.5000000000000000) * z) + (p4 / m))),
 (m,
  select(((p2**2.0000000000000000 + p3**2.0000000000000000 + p4**2.0000000000000000)**0.50000000000000000 == 0.0000000000000000), 0.0000000000000000, (-(p2**2.0000000000000000 + p3**2.0000000000000000 + p4**2.0000000000000000)**0.50000000000000000 / p1)))]

In [12]:
xxx

NameError: name 'xxx' is not defined

The analytical gradient its exact and faster, seems like a no brainer to use it. 

In reality, the effects on the optimization technique used are not straightforward, making the option to use numerical gradients still interesting in some, albeit rare, cases.

## Solving the low-thrust transfer

We define (again) the optimization problem, and set a tolerance for *pagmo* to be able to judge the relative value of two individuals. 

:::{note}
This tolerance has a different role from the numerical tolerance set in the particular algorithm chosen to solve the problem and is only used by the *pagmo* machinery to decide outside the optimizer whether the new proposed indivdual is better than what was the previous *champion*.

In [ ]:
prob_g = pg.problem(udp_g)
prob_g.c_tol = 1e-6

... and we define an optimization algorithm.

In [ ]:
snopt72 = "/Users/harry.holt/opt/libsnopt7_c.dylib"
uda = ppnf.snopt7(library=snopt72, minor_version=2, screen_output=False)
uda.set_integer_option("Major iterations limit", 2000)
uda.set_integer_option("Iterations limit", 20000)
uda.set_numeric_option("Major optimality tolerance", 1e-3)
uda.set_numeric_option("Major feasibility tolerance", 1e-8)

algo = pg.algorithm(uda)

We solve the problem from random initial guess ten times and only save the result if a feasible solution is found (as defined by the criterias above)

# Now convert to a higher number of segments

In [ ]:
nseg_old = nseg

In [ ]:
nseg_max = 80
nseg = nseg_old

while nseg <= nseg_max:

    udp_g = pk.trajopt.direct_cr3bp(
            pls=posvel0,
            plf=posvelf,
            ms=ms,
            mu=mu,
            max_thrust=max_thrust,
            veff=veff,
            t0_bounds=[0.0, 0.0],
            tof_bounds=[tof,tof],
            mf_bounds=[ms*0.5, ms],
            vinfs=0.,
            vinff=0.,
            nseg=nseg,
            cut=cut,
            with_gradient=True,
            high_fidelity=True
    )

    prob_g = pg.problem(udp_g)
    prob_g.c_tol = 1e-6

    if nseg != nseg_old:
        throttles_old = best_x[8:-1]
        throttles_init = [0.0] * (nseg * 3)
        for nn in range(nseg_old):
            throttles_init[nn*6:nn*6+3] = throttles_old[nn*3:nn*3+3]
            throttles_init[nn*6+3:nn*6+6] = throttles_old[nn*3:nn*3+3]
        x_init = best_x[0:8].tolist() + throttles_init + [best_x[-1].item()]
        # x_init = np.array(x_init, dtype=float).tolist()

    print('--------------------------')
    print(f'Testing Nseg {nseg}')
    total_time = 0.0

    masses = []
    xs = []
    for i in range(10):
        time_start = time.time()
        
        if nseg != nseg_old:
            pop_g = pg.population(prob_g, 0)
            pop_g.push_back(x_init)
        else:
            pop_g = pg.population(prob_g, 1)
        
        pop_g = algo.evolve(pop_g)
        
        time_end = time.time()
        total_time += time_end - time_start

        # print('err', np.linalg.norm(pop_g.champion_f[1:]), np.linalg.norm(pop_g.champion_f[1:4]), np.linalg.norm(pop_g.champion_f[4:7]))
        if(prob_g.feasibility_f(pop_g.champion_f)):
            print(".", end="")
            masses.append(pop_g.champion_x[1])
            xs.append(pop_g.champion_x)
            if nseg != nseg_old:
                break
        else:
            print("x", end ="")

    if len(masses)>0:
        print("\nBest mass is: ", np.max(masses))
        print(f"Total time to success: {total_time:.3f} seconds")
        best_idx = np.argmax(masses)
    else:
        xs = [pop_g.champion_x]
        best_idx = 0
        print("\nNo solution found")

    best_x = xs[best_idx]

    # Update number of segments
    nseg = nseg * 2

And we plot the trajectory found:

In [ ]:
udp_g.pretty(best_x)

In [ ]:
ax = udp_g.plot(best_x, show_gridpoints=False, show_throttles=True, show_midpoints=False)
# Making the axes nicer
# ax.set_xlim(-0.1,1.1)
# ax.set_ylim(-1/2,1/2)
# ax.view_init(-90,-90)
# ax.axis('off');

In [ ]:
nseg = 4

In [ ]:
# Extract throttle values from the best solution
tgrid = np.linspace(best_x[0], best_x[-1], nseg+1)
throttle_profile = best_x[8:-1].reshape(nseg, 3)
# throttle_profile_ones = np.ones_like(throttle_profile)

# Objective: integral of the thrust
# obj = np.trapezoid(np.linalg.norm(throttle_profile[:, 0:3],axis=1),tgrid)/(tgrid[-1]-tgrid[0])
obj = np.sum(np.linalg.norm(throttle_profile[:, 0:3],axis=1) * np.diff(tgrid)) / (tgrid[-1]-tgrid[0])

# Create time grid for throttle segments
segment_times = np.linspace(0, tof, nseg)

# Plot thrust profile for each axis
plt.figure(figsize=(8, 5))
plt.step(tgrid, np.hstack((throttle_profile[:, 0], throttle_profile[-1, 0])), label='u_x', where='post')
plt.step(tgrid, np.hstack((throttle_profile[:, 1], throttle_profile[-1, 1])), label='u_y', where='post')
plt.step(tgrid, np.hstack((throttle_profile[:, 2], throttle_profile[-1, 2])), label='u_z', where='post')
plt.step(tgrid, np.hstack((np.linalg.norm(throttle_profile[:, 0:3],axis=1), np.linalg.norm(throttle_profile[-1, 0:3]))), color='k', label='||u||', where='post')
plt.xlabel('Time [-]')
plt.ylabel('Throttle')
plt.title(f'Thrust Profile: Obj {obj:.4f}')
plt.legend()
plt.grid(True)
plt.show()